# Assignment week 6

This assignment is structured in two blocks: individual tasks and group tasks. The individual tasks are made of simple preliminary exercises that should be done individually before the instruction session. These are necessary/helpful for completing the group tasks. The group tasks involve the use of Python/Pyomo and consist of a complete optimization problem, which will be solved during the instructions.

## Individual tasks
### Exercise 1
Given the following problem, you need to install power plants with sufficient capacity to supply demand, at minimum cost

Locations: 1, 2, 3, 4

Power plants: PV, Wind

|Location | PV (MW) | Wind (MW) 	|
|-------|-----------|---------------|
| 1 	| 100     	| 0         	|
| 2 	| 150     	| 0         	|
| 3 	| 400     	| 1000      	|
| 4 	| 0       	| 1000      	|

Costs:
- PV: 500 EUR/kW
- Wind: 700 EUR/kW

Electricity demand:

| Location 	| Demand (MWh) 	|
|-----------|---------------|
| 1        	| 300          	|
| 2        	| 300          	|
| 3        	| 500          	|
| 4        	| 200          	|

The locations are connected with a transmission grid.

There is no transmission loss between the locations.

***1.a)*** Formulate this optimization problem, constraints include:
- Max installed capacity per location and power plant type
- Demand + supply equality

$ \color{red} \textbf{Variables:} $

- $ \color{red} P_{PV,i} \ \ \text{with} \ i=1,2,3,4 \text{: generation of PV per location } i $
- $ \color{red} P_{Wind,i} \ \ \text{with} \ i=1,2,3,4 \text{: generation of Wind per location }i$
- $ \color{red} P_{Demand,i} \ \ \text{with} \ i=1,2,3,4 \text{: Demand per location }i$

$ \color{red} \textbf{Objective function:} $

$ \color{red} \textbf{min } J_{\text{cost}} = \min  \sum_{i=1}^{4} \big ( P_{PV,i} \ C_{PV} + P_{Wind,i} \ C_{Wind} \big )$

$ \ \ \ \color{red} \text{with} \ \ C_{PV} = 0.5 \ \text{EUR}/\text{MW} \ \ \text{and} \ \ C_{Wind} = 0.7 \ \text{EUR}/\text{MW} $


$ \color{red} \textbf{Constraints:} $

- $ \color{red}  \sum_{i=1}^{4} P_{PV,i} + \sum_{i=1}^{4} P_{Wind,i} = \sum_{i=1}^{4} P_{Demand,i} $

- $ \color{red}  P_{PV,i} \leq P_{PV,i}^{\max} \ \ \text{and} \ \ P_{Wind,i} \leq P_{Wind,i}^{\max} \ \ \text{for each} \ i=1,2,3,4 $






***1.b)*** Implement the problem in **Pyomo**

If you are running this on Google Colab, you need to uncomment (remove the `#`) and execute the following lines to install the Pyomo package, the solver, and some helper tools. If you are running this on Binder or elsewhere (e.g. your own computer) you can ignore this.

In [ ]:
# !pip install pyomo==6.4.1
# !pip install "git+https://github.com/sjpfenninger/sen1511.git#egg=optimutils&subdirectory=optimutils"

# !wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
# !chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh
# !bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -f -p /usr/local/
# !conda install -y -c conda-forge ipopt

In [ ]:
import pyomo.environ as pyo

from optimutils import summarise_results

In [ ]:
# Exercise 1b


## Group Tasks

## Exercise 2
Now consider that there is a 10% transmission loss when transporting electricity between locations. The transmission system looks as follows:

```
┌───┐   ┌───┐   ┌───┐
│ 1 ├───┤ 2 ├───┤ 3 │
└───┘   └───┘   └───┘
```

```
                 ┌───┐
           ┌─────┤ 3 │
           │     └───┘
┌───┐    ┌─┴─┐
│ 1 ├────┤ 2 │
└───┘    └─┬─┘
           │     ┌───┐
           └─────┤ 4 │
                 └───┘
```


***2.a)*** How would you formulate this extended problem?


<div style="color:blue">

*not really sure...*

using the following notation:
$P_{G,i}$ is the power generated by both $P_{PV,i}$ and $P_{Wind,i}$ at node $i$.
 $$P_{G,i} = P_{PV,i} + P_{Wind,i}$$

Then for each node the equality constraint will be different according to the transmission grid:

\begin{align}

 P_{demand,1} = & P_{G,1} + 0.9 P_{G,2} + 0.9^2 P_{G,3} + 0.9^2 P_{G,4} \\
 P_{demand,2} = & P_{G,2} + 0.9 P_{G_1} + 0.9 P_{G,3} + 0.9 P_{G,4} \\
 P_{demand,3} = & P_{G,3} + 0.9^2 P_{G_1} + 0.9 P_{G,2} + 0.9^2 P_{G,4} \\
 P_{demand,4} = & P_{G,4} + 0.9^2 P_{G_1} + 0.9 P_{G,2} + 0.9^2 P_{G,3} \\
\end{align}

</div>

In [ ]:
#variables
m = pyo.ConcreteModel(name = "test")

location = [1,2,3,4]
generator = [1,2] #[PV, wind]

m.power_generation = pyo.Var(generator, location, domain= pyo.NonNegativeReals) #P_Wind,i and P_PV,i
# m.power_demand = pyo.Var(location, domain= pyo.NonNegativeReals) #P_demand,i

#constants
generation_prices = [500000,700000] # [C_PV, C_Wind]
power_demand = [300, 300, 500, 200]
#objective function
m.cost_function = pyo.Objective(
    expr= sum(generation_prices[g-1]*m.power_generation[g,i] for g in generator for i in location) ,
    sense = pyo.minimize)

#constraints
generate_unit_max = [[100,150,400,0] , [ 0,0,1000,1000]]

for g in generator: 
  for i in location:
    setattr(m, f"lim_max_{g},{i}", pyo.Constraint(expr =  m.power_generation[g,i] <= generate_unit_max[g-1][i-1]))


m.eq_constraint_1 = pyo.Constraint( expr = power_demand[0] <= sum(m.power_generation[g,1] for g in generator) + 0.9*(sum(m.power_generation[g,2] for g in generator) - power_demand[1]) + 0.81 * (sum(m.power_generation[g,3] for g in generator) - power_demand[2]) + 0.81 * (sum(m.power_generation[g,4] for g in generator)-power_demand[3]))
m.eq_constraint_2 = pyo.Constraint( expr = power_demand[1] <= sum(m.power_generation[g,2] for g in generator) + 0.9*(sum(m.power_generation[g,1] for g in generator) - power_demand[0])+ 0.9*(sum(m.power_generation[g,3] for g in generator) - power_demand[2]) + 0.9 * (sum(m.power_generation[g,4] for g in generator)-power_demand[3]))
m.eq_constraint_3 = pyo.Constraint( expr = power_demand[2] <= sum(m.power_generation[g,3] for g in generator) + 0.81*(sum(m.power_generation[g,1] for g in generator) - power_demand[0])+ 0.9*(sum(m.power_generation[g,2] for g in generator) - power_demand[1]) + 0.81 * (sum(m.power_generation[g,4] for g in generator)-power_demand[3]))
m.eq_constraint_4 = pyo.Constraint( expr = power_demand[3] <= sum(m.power_generation[g,4] for g in generator) + 0.81*(sum(m.power_generation[g,1] for g in generator) - power_demand[0])+ 0.9*(sum(m.power_generation[g,2] for g in generator) - power_demand[1]) + 0.81 * (sum(m.power_generation[g,3] for g in generator)-power_demand[2]))

#solve
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
solver = pyo.SolverFactory('glpk')

solver.solve(m)

summarise_results(m)

In [ ]:
#variables
m = pyo.ConcreteModel(name = "test")

location = [1,2,3,4]
generator = [1,2] #[PV, wind]

m.power_generation = pyo.Var(generator, location, domain= pyo.NonNegativeReals) #P_Wind,i and P_PV,i
# m.power_demand = pyo.Var(location, domain= pyo.NonNegativeReals) #P_demand,i

#constants
generation_prices = [500000,700000] # [C_PV, C_Wind]
power_demand = [300, 300, 500, 200]
#objective function
m.cost_function = pyo.Objective(
    expr= sum(generation_prices[g-1]*m.power_generation[g,i] for g in generator for i in location) ,
    sense = pyo.minimize)

#constraints
generate_unit_max = [[100,150,400,0] , [ 0,0,1000,1000]]

for g in generator: 
  for i in location:
    setattr(m, f"lim_max_{g},{i}", pyo.Constraint(expr =  m.power_generation[g,i] <= generate_unit_max[g-1][i-1]))


m.eq_constraint_1 = pyo.Constraint( expr = power_demand[0] <= sum(m.power_generation[g,1] for g in generator) + 0.5*(sum(m.power_generation[g,2] for g in generator) - power_demand[1]) + 0.25 * (sum(m.power_generation[g,3] for g in generator) - power_demand[2]) + 0.25 * (sum(m.power_generation[g,4] for g in generator)-power_demand[3]))
m.eq_constraint_2 = pyo.Constraint( expr = power_demand[1] <= sum(m.power_generation[g,2] for g in generator) + 0.5*(sum(m.power_generation[g,1] for g in generator) - power_demand[0])+ 0.5*(sum(m.power_generation[g,3] for g in generator) - power_demand[2]) + 0.5 * (sum(m.power_generation[g,4] for g in generator)-power_demand[3]))
m.eq_constraint_3 = pyo.Constraint( expr = power_demand[2] <= sum(m.power_generation[g,3] for g in generator) + 0.25*(sum(m.power_generation[g,1] for g in generator) - power_demand[0])+ 0.5*(sum(m.power_generation[g,2] for g in generator) - power_demand[1]) + 0.25 * (sum(m.power_generation[g,4] for g in generator)-power_demand[3]))
m.eq_constraint_4 = pyo.Constraint( expr = power_demand[3] <= sum(m.power_generation[g,4] for g in generator) + 0.25*(sum(m.power_generation[g,1] for g in generator) - power_demand[0])+ 0.5*(sum(m.power_generation[g,2] for g in generator) - power_demand[1]) + 0.25 * (sum(m.power_generation[g,3] for g in generator)-power_demand[2]))


# m.limits = pyo.Constraint(expr = sum(m.power_generation[g,i] for g in generator for i in location) == sum(power_demand[i-1] for i in location))

#solve
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
solver = pyo.SolverFactory('glpk')

solver.solve(m)

summarise_results(m)

***2.b)*** Implement this variant of the problem using Calliope